In [1]:
import pandas as pd
import urllib.request, json
import plotly_express as px
import plotly.figure_factory as ff
from pyxirr import xirr

In [3]:
import yfinance as yf

In [ ]:
with urllib.request.urlopen('https://api.mfapi.in/mf') as url:
    data = json.load(url)
df_mfs = pd.DataFrame(data)

In [ ]:
df_mfs[df_mfs.schemeName.str.contains('HDFC Flex')]

In [ ]:
df_mfs[df_mfs.schemeName.str.startswith('Parag')]

In [ ]:
df_sel_name = df_mfs.schemeName[0]

In [ ]:
df_sel_code = df_mfs[df_mfs['schemeName'] == df_sel_name]['schemeCode'][0]
df_sel_code

In [ ]:
def get_nav(scheme_code = '122639'):
    # scheme_code = '122639'
    mf_url = 'https://api.mfapi.in/mf/' + scheme_code
    with urllib.request.urlopen(mf_url) as url:
        data = json.load(url)

    df_navs = pd.DataFrame(data['data'])
    df_navs['date'] = pd.to_datetime(df_navs.date, format='%d-%m-%Y')
    df_navs['nav'] = df_navs['nav'].astype(float)
    df_navs = df_navs.sort_values(['date']).set_index(['date'])
    df_dates = pd.DataFrame(pd.date_range(start=df_navs.index.min(), end=df_navs.index.max()), columns=['date']).set_index(['date'])
    df_navs = df_navs.join(df_dates, how='outer').ffill().reset_index()
    return df_navs

In [ ]:
df_navs = get_nav()

In [ ]:
base_date = '2015-01-01'
base_nav = df_navs.set_index(['date']).loc[base_date:].nav[0]
df_sel_navs = df_navs[df_navs['date'] >= base_date].copy()
df_sel_navs

In [ ]:
px.line(df_navs, x = 'date', y='nav', log_y=True)

In [ ]:
def get_cagr(df_navs_orig, num_years = 1):
    df_navs = df_navs_orig.copy()
    df_navs['prev_nav'] = df_navs.nav.shift(365 * num_years)
    df_navs = df_navs.dropna()
    df_navs['returns'] = df_navs['nav'] / df_navs['prev_nav'] - 1
    df_navs['cagr'] = 100 * ((1 + df_navs['returns']) ** (1 / num_years) - 1)
    df_navs['years'] = num_years
    df_cagr = df_navs[['date', 'years', 'cagr']]
    return df_cagr

In [ ]:
all_names = ['Parag Parikh Flexi Cap Fund - Direct Plan - Growth', 'HDFC Equity Fund - Growth Option']

In [ ]:
list_navs = []
for name in all_names:
    code = df_mfs[df_mfs['schemeName'] == name].schemeCode.to_list()[0]
    df_nav_comp = get_nav(str(code))
    df_nav_comp = df_nav_comp.set_index('date')
    df_nav_comp = df_nav_comp.rename(columns={'nav': name})
    list_navs.append(df_nav_comp)

df_nav_all = pd.concat(list_navs, axis=1).dropna()
df_rebased = df_nav_all.div(df_nav_all.iloc[0]).reset_index()
df_rebased_long = pd.melt(df_rebased, id_vars='date', value_vars=all_names, var_name='mf', value_name='nav')

In [ ]:
df_nav = df_nav_comp.reset_index()
df_nav.columns = ['date', 'nav']

In [ ]:
df_nav.dtypes

In [ ]:
df_nav.head()

In [ ]:
start_date = pd.to_datetime('2006-05-01')
end_date = pd.to_datetime('2020-04-01')

In [ ]:
df_nav[df_nav['date'] >= start_date].head()

In [ ]:
df_dates = pd.DataFrame(pd.date_range(start=start_date, end=end_date, freq='M'))
df_dates.columns = ['date']

In [ ]:
df_cf = df_nav.merge(df_dates, on='date')
df_cf['amount'] = 1000
df_cf['units'] = df_cf['amount'] / df_cf['nav']

In [ ]:
df_cf['cum_units'] = df_cf['units'].cumsum()
df_cf['inv_value'] = df_cf['cum_units'] * df_cf['nav']
df_cf['cum_amount'] = df_cf['amount'].cumsum()

In [ ]:
df_daily_dates = pd.DataFrame(pd.date_range(start=df_cf['date'].min(), end=df_cf['date'].max(), freq='D'))
df_daily_dates.columns = ['date']
df_cfs = df_cf.merge(df_daily_dates, on='date', how='right').sort_values(['date'])
df_cfs = df_cfs.ffill()
df_cf_long = pd.melt(df_cfs[['date', 'cum_amount', 'inv_value']], id_vars=['date'], value_vars=['cum_amount', 'inv_value'])

In [ ]:
px.line(df_cf_long, x='date', y='value', color='variable')

In [ ]:
df_investment = df_cf[['date', 'amount']]

In [ ]:
df_redemption = pd.DataFrame([{'date': df_cf.iloc[-1:].date.values[0], 'amount': -df_cf['units'].sum() * df_cf.iloc[-1:].nav.values[0]}])

In [ ]:
df_irr = pd.concat([df_investment, df_redemption]).reset_index(drop=True)
df_irr

In [ ]:
xirr(df_irr[['date', 'amount']]) * 100

In [ ]:
float('293.323')

In [ ]:
years = [x + 1 for x in range(9)]
list_cagr = []
for y in years:
    df_cagr = get_cagr(df_navs, y)
    list_cagr.append(df_cagr)
df_cagrs = pd.concat(list_cagr)

In [ ]:
df_cagrs.dtypes

In [ ]:
import datetime
import numpy as np

In [ ]:
np.datetime64(datetime.datetime(2002, 6, 28, 1, 0))

In [ ]:
px.line(df_cagrs, x='date', y='cagr', color='years')

In [ ]:
df_cagrs.head()

In [ ]:
dfx = df_cagrs[['date', 'years', 'cagr']].groupby('years').describe().reset_index()
dfx.columns = [[a for (a, b) in dfx.columns][0]] + [a for a in dfx.columns.droplevel()][1:]
dfx

In [ ]:
df_cagr

In [ ]:
all_names

In [ ]:
list_navs = []
list_cagrs = []
for name in all_names:
    code = df_mfs[df_mfs['schemeName'] == name].schemeCode.to_list()[0]
    df_nav_comp = get_nav(str(code))
    years = [x for x in range(1, 11)]
    list_cagr = []
    for y in years:
        df_cagr = get_cagr(df_nav_comp, y)
        list_cagr.append(df_cagr)
    df_cagrs_comp = pd.concat(list_cagr)

    df_nav_comp = df_nav_comp.set_index('date')
    df_nav_comp = df_nav_comp.rename(columns={'nav': name})
    list_navs.append(df_nav_comp)

    df_cagrs_comp = df_cagrs_comp.set_index(['date', 'years'])
    df_cagrs_comp = df_cagrs_comp.rename(columns={'cagr': name})
    list_cagrs.append(df_cagrs_comp)

df_nav_all = pd.concat(list_navs, axis=1).dropna()
df_cagr_all = pd.concat(list_cagrs, axis=1).dropna()

In [ ]:
df_nav_all

In [ ]:
df_cagr_all

In [ ]:
df_navs_date = df_nav_all.reset_index()
min_date = df_navs_date['date'].min()
max_date = df_navs_date['date'].max()
from_date = "2020-03-23"
df_nav_all = df_navs_date[df_navs_date['date'] >= np.datetime64(from_date)].set_index('date')

In [ ]:
df_rebased = df_nav_all.div(df_nav_all.iloc[0]).reset_index()
df_rebased_long = pd.melt(df_rebased, id_vars='date', value_vars=all_names, var_name='mf', value_name='nav')

In [ ]:
df_rebased

In [ ]:
df_cagr_wide = df_cagr_all.reset_index()
df_cagr_long = pd.melt(df_cagr_wide, id_vars=['date', 'years'], value_vars=all_names, var_name='mf', value_name='cagr')

In [ ]:
df_cagr_long

In [ ]:
px.density_contour(df_cagrs.set_index(['date']), x='cagr', color='years')

In [ ]:
# df_cagrs[df_cagrs['years'] == 9]
px.histogram(df_cagrs, x='cagr', color='years')

In [ ]:
df_cagrs['cagr_bucket'] = round(df_cagrs['cagr'] / 1, 0) * 1

In [ ]:
df_total = pd.DataFrame(df_cagrs.groupby(['years']).cagr.count()).reset_index()
df_total.columns = ['years', 'total']

In [ ]:
df_counts = pd.DataFrame(df_cagrs.groupby(['years', 'cagr_bucket']).cagr.count()).reset_index()
df_counts.columns = ['years', 'cagr_bucket', 'count']

In [ ]:
df_x = df_counts.merge(df_total, on='years', how='left')
df_x['pct'] = df_x['count'] / df_x['total']

In [ ]:
df_x['years'] = df_x['years'].astype(str)
px.bar(df_x, x='cagr_bucket', y='pct', color='years', barmode='overlay')

In [ ]:
df_x

In [ ]:
with open('../data/mf_codes.txt', 'r') as fp:
    list_code = []
    line = fp.readline()
    while line:
        words = line.strip().split(';')
        if len(words) > 5:
            list_code.append([words[i] for i in [0, 1, 3]])
        line = fp.readline()

In [ ]:
df_codes = pd.DataFrame(list_code)

In [ ]:
# df_codes.columns = ['code', 'isin', 'none', 'name', 'nav', 'date']
df_codes.columns = ['schemeCode', 'schemeISIN', 'schemeName']

In [ ]:
with urllib.request.urlopen('https://api.mfapi.in/mf') as url:
    data = json.load(url)
df_mfs = pd.DataFrame(data)

In [ ]:
# SWP in progress

In [83]:
list_scr = pd.read_html('https://www.screener.in/company/LGBBROSLTD/consolidated/')

In [84]:
len(list_scr)

10

In [85]:
list_scr[0]

,Unnamed: 0,Dec 2019,Mar 2020,Jun 2020,Sep 2020,Dec 2020,Mar 2021,Jun 2021,Sep 2021,Dec 2021,Mar 2022,Jun 2022,Sep 2022,Dec 2022
0,Sales +,407,356,195,417,498,499,393,577,574,559,521,578,581
1,Expenses +,351,322,180,359,407,410,331,470,458,454,436,476,474
2,Operating Profit,55,34,14,58,91,89,61,107,116,105,85,102,107
3,OPM %,14%,10%,7%,14%,18%,18%,16%,18%,20%,19%,16%,18%,18%
4,Other Income +,6,8,1,6,2,12,4,20,3,8,13,6,7
5,Interest,4,3,3,3,2,2,2,2,3,3,1,1,2
6,Depreciation,20,21,20,21,21,21,20,21,21,21,20,20,20
7,Profit before tax,37,18,-8,41,70,78,43,104,95,90,77,87,93
8,Tax %,25%,30%,25%,33%,23%,25%,23%,25%,25%,29%,24%,23%,22%
9,Net Profit,28,13,-6,27,54,58,33,78,71,64,58,67,72


In [144]:
df_qly_sales = list_scr[0].rename({'Unnamed: 0': 'Metric'}, axis=1)
df_yly_sales = list_scr[1].rename({'Unnamed: 0': 'Metric'}, axis=1)

In [145]:
df_yly_sales

,Metric,Mar 2011,Mar 2012,Mar 2013,Mar 2014,Mar 2015,Mar 2016,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,TTM
0,Sales +,704,898,939,1085,1151,1175,1258,1418,1688,1543,1609,2102,2239
1,Expenses +,616,794,850,959,1009,1037,1095,1222,1479,1355,1355,1713,1840
2,Operating Profit,87,104,89,126,142,138,162,196,209,187,254,389,399
3,OPM %,12%,12%,10%,12%,12%,12%,13%,14%,12%,12%,16%,19%,18%
4,Other Income +,2,2,4,5,9,4,10,5,15,22,21,35,35
5,Interest,15,18,23,18,18,18,16,12,13,16,11,9,7
6,Depreciation,26,29,30,33,39,46,53,58,68,79,83,83,80
7,Profit before tax,48,59,40,80,94,78,103,131,143,115,180,332,347
8,Tax %,4%,25%,19%,19%,22%,17%,28%,33%,32%,22%,26%,26%,NaN
9,Net Profit,46,44,33,66,74,66,75,89,100,91,133,246,261


In [146]:
df_qly_sales.Metric[3]

'OPM %'

In [124]:
df_qly = df_qly_sales[df_qly_sales['Metric'] == df_qly_sales.Metric[0]].transpose()
df_qly = df_qly.reset_index()
df_qly.columns = ['Date', 'Sales (Cr)']
df_qly = df_qly[df_qly['Date'] != 'Metric']
df_qly['Sales (Cr)'] = df_qly['Sales (Cr)'].astype(float)

In [134]:
px.bar(df_qly, x='Date', y='Sales (Cr)', title='Quarterly Sales')

In [136]:
df_qly['QoQ Growth (%)'] = df_qly['Sales (Cr)'].pct_change() * 100
df_qly['YoY Growth (%)'] = df_qly['Sales (Cr)'].pct_change(4) * 100

In [142]:
px.bar(df_qly, x='Date', y='QoQ Growth (%)', title='Quarterly Sales (QoQ)')

In [143]:
px.bar(df_qly.dropna(), x='Date', y='YoY Growth (%)', title='Quarterly Sales (YoY)')

In [3]:
help(yf.download)

Help on function download in module yfinance.multi:

download(tickers, start=None, end=None, actions=False, threads=True, group_by='column', auto_adjust=False, back_adjust=False, progress=True, period='max', show_errors=True, interval='1d', prepost=False, proxy=None, rounding=False, timeout=None, **kwargs)
    Download yahoo tickers
    :Parameters:
        tickers : str, list
            List of tickers to download
        period : str
            Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
            Either Use period parameter or use start and end
        interval : str
            Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            Intraday data cannot extend last 60 days
        start: str
            Download start date string (YYYY-MM-DD) or _datetime.
            Default is 1900-01-01
        end: str
            Download end date string (YYYY-MM-DD) or _datetime.
            Default is now
        group_by : str
            Group by 'ticker' o

In [4]:
infy = yf.Ticker('INFY.NS')

In [4]:
msft = yf.Ticker('MSFT')

In [5]:
infy.actions

,Dividends,Stock Splits
Date,,
1996-04-30 00:00:00+05:30,0.006836,0.0
1996-10-28 00:00:00+05:30,0.002930,0.0
1997-05-23 00:00:00+05:30,0.007813,0.0
1997-08-19 00:00:00+05:30,0.000000,2.0
1997-12-17 00:00:00+05:30,0.005859,0.0
...,...,...
2020-10-23 00:00:00+05:30,12.000000,0.0
2021-05-31 00:00:00+05:30,15.000000,0.0
2021-10-26 00:00:00+05:30,15.000000,0.0


In [6]:
infy.splits

Date
1997-08-19 00:00:00+05:30    2.0
1999-02-10 00:00:00+05:30    2.0
2000-01-24 00:00:00+05:30    2.0
2000-01-27 00:00:00+05:30    2.0
2004-07-01 00:00:00+05:30    4.0
2006-07-13 00:00:00+05:30    2.0
2014-12-02 00:00:00+05:30    2.0
2015-06-15 00:00:00+05:30    2.0
2018-09-04 00:00:00+05:30    2.0
Name: Stock Splits, dtype: float64

In [7]:
infy.dividends

Date
1996-04-30 00:00:00+05:30     0.006836
1996-10-28 00:00:00+05:30     0.002930
1997-05-23 00:00:00+05:30     0.007813
1997-12-17 00:00:00+05:30     0.005859
1998-04-30 00:00:00+05:30     0.017578
1998-11-12 00:00:00+05:30     0.009766
1999-05-20 00:00:00+05:30     0.039063
1999-10-28 00:00:00+05:30     0.023438
2000-04-28 00:00:00+05:30     0.046875
2000-10-25 00:00:00+05:30     0.039063
2001-04-27 00:00:00+05:30     0.117188
2001-10-18 00:00:00+05:30     0.117188
2002-05-21 00:00:00+05:30     0.195313
2002-10-30 00:00:00+05:30     0.195313
2003-05-28 00:00:00+05:30     0.226563
2003-10-16 00:00:00+05:30     0.226563
2004-05-26 00:00:00+05:30     1.562500
2004-10-18 00:00:00+05:30     0.312500
2005-06-01 00:00:00+05:30     0.406250
2005-10-17 00:00:00+05:30     0.406250
2006-05-25 00:00:00+05:30     2.406250
2006-10-19 00:00:00+05:30     0.625000
2007-06-06 00:00:00+05:30     0.812500
2007-10-18 00:00:00+05:30     0.750000
2008-05-29 00:00:00+05:30     3.406250
2008-10-16 00:00:00+

In [8]:
import yahoo_fin.stock_info as si

In [32]:
bl = si.get_balance_sheet('nflx')

In [36]:
msft.info.keys()

dict_keys(['regularMarketPrice', 'preMarketPrice', 'logo_url'])

In [9]:
yf.__version__

'0.2.12'

In [10]:
from yahooquery import Ticker

In [47]:
holdings_file = 'holdings (3).csv'

In [48]:
df_holdings = pd.read_csv(holdings_file)

In [49]:
scrip_codes = [x + '.NS' for x in df_holdings.Instrument.tolist()]

In [50]:
ticker = Ticker(scrip_codes)

In [41]:
income_statement = ticker.income_statement('q') 

In [59]:
df_fin = pd.DataFrame(ticker.financial_data)

In [65]:
df_metrics = df_fin.transpose().reset_index().rename(columns={'index': 'Instrument'})
df_holdings['Instrument'] = df_holdings['Instrument'] + '.NS'

In [68]:
df_holds = df_holdings.merge(df_metrics, on='Instrument')

In [69]:
df_holds.to_csv('holdings.csv')

In [24]:
ticker.earnings_trend

{'ltim.ns': 'No fundamentals data found for any of the summaryTypes=earningsTrend'}

In [25]:
ticker.earnings

{'ltim.ns': 'No fundamentals data found for any of the summaryTypes=earnings'}